In [1]:
from google.colab import files
uploaded = files.upload()

Saving MasterResearchFunction.py to MasterResearchFunction.py


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install japanize_matplotlib bottleneck tslearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.1/354.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 27.3 MB/s eta 0:00:00
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=a153f9928120f42e4615de6291737a23fca9d1fd5ecd73549aa44e1bfb9918ed
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
Successfully built japanize_matplotlib


In [4]:
#作った関数
import MasterResearchFunction as mr
# 基本ライブラリ
import os, re, csv, math, statistics
from datetime import datetime, timedelta
from decimal import Decimal

# 数値計算とデータ処理
import numpy as np
import pandas as pd
import bottleneck as bn

# 機械学習ライブラリ
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# ディープラーニングライブラリ
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation, Conv1D, MaxPooling1D, Flatten

# プロットと可視化
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import japanize_matplotlib

# その他のライブラリ
from scipy import signal, stats
from scipy.signal import savgol_filter
from tslearn.metrics import dtw_path

In [5]:
#Apple Watchのモーションデータの読み込み
#a_yuuma_motion_data = mr.process_apple_watch_csv("datasets/yuuma/20240604/yuuma_motion.csv")
#b_sakamoto_motion_data = mr.process_apple_watch_csv("datasets/sakamoto/20240604/sakamoto_motion.csv")
#c_watabe_motion_data = mr.process_apple_watch_csv("datasets/watabe/20240605/watabe_motion.csv")
#d_nakazawa_motion_data = mr.process_apple_watch_csv("datasets/nakazawa/20240606/nakazawa_motion.csv")
#e_okede_motion_data = mr.process_apple_watch_csv("datasets/okeda/20240607/okeda_motion.csv")
a_yuuma_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/yuuma/20240604/yuuma_motion.csv")
b_sakamoto_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/sakamoto/20240604/sakamoto_motion.csv")
c_watabe_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/watabe/20240605/watabe_motion.csv")
d_nakazawa_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/nakazawa/20240606/nakazawa_motion.csv")
e_okede_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/okeda/20240607/okeda_motion.csv")

In [6]:
#Tobiiのモーションデータの読み込み
#a_yuuma_eye_data = mr.process_tobii_csv("datasets/yuuma/20240604/yuuma_eye.csv")
#b_sakamoto_eye_data = mr.process_tobii_csv("datasets/sakamoto/20240604/sakamoto_eye.csv")
#c_watabe_eye_data = mr.process_tobii_csv("datasets/watabe/20240605/watabe_eye.csv")
#d_nakazawa_eye_data = mr.process_tobii_csv("datasets/nakazawa/20240606/nakazawa_eye.csv")
#e_okede_eye_data = mr.process_tobii_csv("datasets/okeda/20240607/okeda_eye.csv")
a_yuuma_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/yuuma/20240604/yuuma_eye.csv")
b_sakamoto_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/sakamoto/20240604/sakamoto_eye.csv")
c_watabe_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/watabe/20240605/watabe_eye.csv")
d_nakazawa_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/nakazawa/20240606/nakazawa_eye.csv")
e_okede_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/okeda/20240607/okeda_eye.csv")

/content/MasterResearchFunction.py:51: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  eye_data = pd.read_csv(file_path, header=0)
/content/MasterResearchFunction.py:51: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  eye_data = pd.read_csv(file_path, header=0)
/content/MasterResearchFunction.py:51: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  eye_data = pd.read_csv(file_path, header=0)
/content/MasterResearchFunction.py:51: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  eye_data = pd.read_csv(file_path, header=0)


In [7]:
#a_yuumaくんの教師データ読み込み
# a_yuuma_check = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/check")
# a_yuuma_circle = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/circle")
# a_yuuma_cross = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/cross")
# a_yuuma_tri = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/tri")
a_yuuma_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/check")
a_yuuma_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/circle")
a_yuuma_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/cross")
a_yuuma_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/tri")

In [8]:
#b_sakamotoくんの教師データ読み込み
# b_sakamoto_check = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture/check")
# b_sakamoto_circle = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture/circle")
# b_sakamoto_cross = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture/cross")
# b_sakamoto_tri = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture/tri")
b_sakamoto_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture/check")
b_sakamoto_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture/circle")
b_sakamoto_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture/cross")
b_sakamoto_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture/tri")

In [ ]:
#c_watabeくんの教師データ読み込み
# c_watabe_check = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/check")
# c_watabe_circle = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/circle")
# c_watabe_cross = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/cross")
# c_watabe_tri = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/tri")
c_watabe_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/check")
c_watabe_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/circle")
c_watabe_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/cross")
c_watabe_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/tri")

In [ ]:
#d_nakazawaくんの教師データ読み込み
# d_nakazawa_check = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/check")
# d_nakazawa_circle = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/circle")
# d_nakazawa_cross = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/cross")
# d_nakazawa_tri = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/tri")
d_nakazawa_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/check")
d_nakazawa_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/circle")
d_nakazawa_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/cross")
d_nakazawa_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/tri")

In [9]:
#e_okedaくんの教師データ読み込み
# e_okeda_check = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/check")
# e_okeda_circle = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/circle")
# e_okeda_cross = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/cross")
# e_okeda_tri = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/tri")
e_okeda_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/check")
e_okeda_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/circle")
e_okeda_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/cross")
e_okeda_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/tri")

In [10]:
Th = 8
# segx, segy, segz = [], [], []

In [11]:
a_yuuma_circle_segx, a_yuuma_circle_segy, a_yuuma_circle_segz = mr.three_axis_spring_ogawa(a_yuuma_motion_data, a_yuuma_circle, Th)
# a_yuuma_cross_segx, a_yuuma_cross_segy, a_yuuma_cross_segz = mr.three_axis_spring_ogawa(a_yuuma_motion_data, a_yuuma_cross, Th)
# a_yuuma_tri_segx, a_yuuma_tri_segy, a_yuuma_tri_segz = mr.three_axis_spring_ogawa(a_yuuma_motion_data, a_yuuma_tri, Th)
# a_yuuma_check_segx, a_yuuma_check_segy, a_yuuma_check_segz = mr.three_axis_spring_ogawa(a_yuuma_motion_data, a_yuuma_check, Th)

ストリーミング出力は最後の 5000 行に切り捨てられました。
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
***

In [12]:
# 各教師データの結果ごとにオーバーラップを検出
def overlap_from_three_segments(segx, segy, segz):
  overlap = []
  for i in range(len(segx)):
    overlap.append(mr.combine_and_find_overlapping_segments(segx[i], segy[i], segz[i]))
  return overlap

In [13]:
a_yuuma_circle_overlap = overlap_from_three_segments(a_yuuma_circle_segx, a_yuuma_circle_segy, a_yuuma_circle_segz)

In [32]:
# 各教師データの結果ごとに出したオーバーラップから、条件を満たさないセグメントを削除する関数
def filter_overlap_by_elapsed_time(overlap, motion_data, min_time=2, max_time=5):
    filtered_overlap = []
    for segments in overlap:
        temp = []
        for start, end in segments:
            start_time = motion_data['Timestamp'][start]
            end_time = motion_data['Timestamp'][end]
            elapsed_time = (end_time - start_time).total_seconds()
            if min_time < elapsed_time < max_time:
                temp.append((start, end))
        if temp:  # フィルタリング後にデータが存在する場合のみ追加
            filtered_overlap.append(temp)
    return filtered_overlap

In [33]:
# 経過時間のフィルタリングを適用
filtered_overlap = filter_overlap_by_elapsed_time(a_yuuma_circle_overlap, a_yuuma_motion_data)

In [24]:
filtered_overlap

[[(10, 125),
  (374, 551),
  (809, 984),
  (1251, 1427),
  (3264, 3432),
  (4388, 4531),
  (4601, 4741),
  (5937, 6058),
  (6198, 6300),
  (6901, 7002),
  (7709, 7950),
  (8046, 8154),
  (8217, 8394),
  (9010, 9135),
  (9446, 9554),
  (9555, 9724),
  (9806, 9947),
  (10212, 10358),
  (11344, 11450),
  (11518, 11692),
  (11828, 11959),
  (12040, 12157),
  (12300, 12407),
  (12513, 12686),
  (12740, 12899),
  (13451, 13587),
  (13961, 14094),
  (14095, 14231),
  (14341, 14458),
  (14653, 14866),
  (14890, 15016),
  (15595, 15714),
  (15959, 16188),
  (16388, 16518),
  (17162, 17357),
  (17958, 18134),
  (18243, 18347),
  (18423, 18540),
  (18564, 18673),
  (18704, 18846),
  (19263, 19481),
  (20408, 20530),
  (20852, 20991),
  (21088, 21189),
  (21391, 21524),
  (21541, 21665),
  (21803, 21924),
  (21985, 22157),
  (22457, 22586),
  (22669, 22779),
  (22961, 23093),
  (23508, 23611),
  (23632, 23867),
  (24825, 25023),
  (25036, 25193),
  (25686, 25840),
  (25964, 26082),
  (26544, 26651

In [27]:
#短くなるやつ
def combine_all_overlaps(overlap):
    # Combine all segments from multiple lists into one list
    combined_segments = []
    for segments in overlap:
        combined_segments.extend(segments)

    # Sort combined segments
    combined_segments.sort()

    # Find and merge overlapping segments
    final_segments = []
    current_overlap = None

    for start, end in combined_segments:
        if current_overlap is None:
            current_overlap = (start, end)
        else:
            current_start, current_end = current_overlap
            if start <= current_end:
                current_overlap = (current_start, max(current_end, end))
            else:
                final_segments.append(current_overlap)
                current_overlap = (start, end)

    if current_overlap is not None:
        final_segments.append(current_overlap)

    return final_segments

In [34]:
combine_overlap = combine_all_overlaps(filtered_overlap)

In [109]:
len(combine_overlap)

231

In [42]:
a_yuuma_circle_overlap = combine_overlap

In [43]:
def extract_timestamp_from_overlap(motion_data, combine_overlap):
  results = []
  for start, end in combine_overlap:
      a = [motion_data['Timestamp'][start], motion_data['Timestamp'][end]]
      results.append(a)
  return results

In [44]:
results =  extract_timestamp_from_overlap(a_yuuma_motion_data, combine_overlap)

In [108]:
results

[[Timestamp('2024-06-04 15:57:16.124933888'),
  Timestamp('2024-06-04 15:57:21.421780992')],
 [Timestamp('2024-06-04 15:57:23.300600064'),
  Timestamp('2024-06-04 15:57:26.938594048')],
 [Timestamp('2024-06-04 15:57:31.335976960'),
  Timestamp('2024-06-04 15:57:35.593611008')],
 [Timestamp('2024-06-04 15:57:36.033399040'),
  Timestamp('2024-06-04 15:57:38.611918848')],
 [Timestamp('2024-06-04 15:57:39.691267072'),
  Timestamp('2024-06-04 15:57:44.448593152')],
 [Timestamp('2024-06-04 15:58:04.776975104'),
  Timestamp('2024-06-04 15:58:08.374782976')],
 [Timestamp('2024-06-04 15:58:08.494770176'),
  Timestamp('2024-06-04 15:58:10.653780992')],
 [Timestamp('2024-06-04 15:58:20.008302080'),
  Timestamp('2024-06-04 15:58:24.525927168')],
 [Timestamp('2024-06-04 15:58:25.145885952'),
  Timestamp('2024-06-04 15:58:28.303538944')],
 [Timestamp('2024-06-04 15:58:28.543521792'),
  Timestamp('2024-06-04 15:58:32.341177856')],
 [Timestamp('2024-06-04 15:58:32.781085952'),
  Timestamp('2024-06-04 

In [49]:
# 経過時間を算出して条件に合わないセグメントを削除する関数
def filter_segments_by_time_length(results, min_time=2, max_time=5):
    filtered_results = []
    for start_time, end_time in results:
        elapsed_time = (end_time - start_time).total_seconds()
        if min_time < elapsed_time < max_time:
            filtered_results.append([start_time, end_time])
    return filtered_results

In [50]:
# 各組の時間の長さが2秒以下と5秒以上のものを弾く
filtered_results = filter_segments_by_time_length(results)

In [98]:
# 各組の時間の長さが2秒以下と5秒以上のものを弾く
len(filtered_results)

174

In [66]:
filtered_results

[[Timestamp('2024-06-04 15:57:23.300600064'),
  Timestamp('2024-06-04 15:57:26.938594048')],
 [Timestamp('2024-06-04 15:57:31.335976960'),
  Timestamp('2024-06-04 15:57:35.593611008')],
 [Timestamp('2024-06-04 15:57:36.033399040'),
  Timestamp('2024-06-04 15:57:38.611918848')],
 [Timestamp('2024-06-04 15:57:39.691267072'),
  Timestamp('2024-06-04 15:57:44.448593152')],
 [Timestamp('2024-06-04 15:58:04.776975104'),
  Timestamp('2024-06-04 15:58:08.374782976')],
 [Timestamp('2024-06-04 15:58:08.494770176'),
  Timestamp('2024-06-04 15:58:10.653780992')],
 [Timestamp('2024-06-04 15:58:20.008302080'),
  Timestamp('2024-06-04 15:58:24.525927168')],
 [Timestamp('2024-06-04 15:58:25.145885952'),
  Timestamp('2024-06-04 15:58:28.303538944')],
 [Timestamp('2024-06-04 15:58:28.543521792'),
  Timestamp('2024-06-04 15:58:32.341177856')],
 [Timestamp('2024-06-04 15:58:32.781085952'),
  Timestamp('2024-06-04 15:58:36.179410944')],
 [Timestamp('2024-06-04 15:58:36.199235840'),
  Timestamp('2024-06-04 

In [99]:
# filtered_results の範囲内の eye_data['Timestamp'] を抽出する関数
def extract_eye_data_within_intervals(filtered_results, eye_data):
    extracted_eye_data = []
    for start_time, end_time in filtered_results:
        # タイムスタンプが範囲内のデータを抽出
        extracted_eye_data.append(eye_data[(eye_data['Timestamp'] >= start_time) & (eye_data['Timestamp'] <= end_time)])

    return extracted_eye_data

In [100]:
# タイムスタンプの範囲内のデータを抽出
extracted_eye_data = extract_eye_data_within_intervals(filtered_results, a_yuuma_eye_data)

In [113]:
len(extracted_eye_data)

174

In [166]:
def filter_by_variance(extracted_eye_data, filtered_results, threshold):
    filtered_extracted_eye_data = []
    filtered_filtered_results = []

    for i in range(len(extracted_eye_data)):
        diff_x = extracted_eye_data[i]['Gaze point X'].diff().fillna(0)
        diff_y = extracted_eye_data[i]['Gaze point Y'].diff().fillna(0)
        distance = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離
        variance = distance.var()  # ユークリッド距離の分散を計算

        if variance < threshold:
            filtered_extracted_eye_data.append(extracted_eye_data[i])
            filtered_filtered_results.append(filtered_results[i])

    return filtered_extracted_eye_data, filtered_filtered_results

In [175]:
Th = 360

In [176]:
# 分散を計算し、閾値を上回る区間を削除
filtered_extracted_eye_data, filtered_filtered_results = filter_by_variance(extracted_eye_data, filtered_results, Th)

In [177]:
len(filtered_filtered_results)

43

In [178]:
filtered_filtered_results

[[Timestamp('2024-06-04 15:58:52.310231040'),
  Timestamp('2024-06-04 15:58:54.388977152')],
 [Timestamp('2024-06-04 16:00:34.852168192'),
  Timestamp('2024-06-04 16:00:37.010668032')],
 [Timestamp('2024-06-04 16:06:22.193654016'),
  Timestamp('2024-06-04 16:06:26.990992128')],
 [Timestamp('2024-06-04 16:07:38.509497856'),
  Timestamp('2024-06-04 16:07:41.308494080')],
 [Timestamp('2024-06-04 16:08:03.835232'),
  Timestamp('2024-06-04 16:08:07.912768768')],
 [Timestamp('2024-06-04 16:08:08.272454912'),
  Timestamp('2024-06-04 16:08:12.088804864')],
 [Timestamp('2024-06-04 16:22:28.643273728'),
  Timestamp('2024-06-04 16:22:30.722300928')],
 [Timestamp('2024-06-04 16:23:32.229618944'),
  Timestamp('2024-06-04 16:23:36.646996992')],
 [Timestamp('2024-06-04 16:23:50.159789056'),
  Timestamp('2024-06-04 16:23:52.418658048')],
 [Timestamp('2024-06-04 16:24:30.058226944'),
  Timestamp('2024-06-04 16:24:34.535913984')],
 [Timestamp('2024-06-04 16:24:37.414481152'),
  Timestamp('2024-06-04 16:

In [249]:
len(filtered_filtered_results[0])

10

In [360]:
start_time = '2024-06-04 19:21:52'
end_time = '2024-06-04 19:21:55'

In [361]:
filtered_data = eye_data[(eye_data['Timestamp'] >= start_time) & (eye_data['Timestamp'] <= end_time)]

In [363]:
filtered_data['Gaze point X'].diff().var()

1284.199925037481

In [365]:
filtered_data['Pupil diameter right'].diff().var()

0.015155823010323003

In [314]:
filtered_data.columns

Index(['Sensor', 'Participant name', 'Event', 'Event value', 'Gaze point X',
       'Gaze point Y', 'Gaze point 3D X', 'Gaze point 3D Y', 'Gaze point 3D Z',
       'Gaze direction left X', 'Gaze direction left Y',
       'Gaze direction left Z', 'Gaze direction right X',
       'Gaze direction right Y', 'Gaze direction right Z',
       'Pupil position left X', 'Pupil position left Y',
       'Pupil position left Z', 'Pupil position right X',
       'Pupil position right Y', 'Pupil position right Z',
       'Pupil diameter left', 'Pupil diameter right', 'Validity left',
       'Validity right', 'Eye movement type', 'Gaze event duration',
       'Eye movement type index', 'Fixation point X', 'Fixation point Y',
       'Ungrouped', 'Timestamp'],
      dtype='object')

In [254]:
filtered_filtered_results

[[[Timestamp('2024-06-04 18:32:57.479557120'),
   Timestamp('2024-06-04 18:32:59.039154944')],
  [Timestamp('2024-06-04 18:33:59.747952896'),
   Timestamp('2024-06-04 18:34:01.406790144')],
  [Timestamp('2024-06-04 18:37:49.707787008'),
   Timestamp('2024-06-04 18:37:54.285170176')],
  [Timestamp('2024-06-04 18:40:52.987462912'),
   Timestamp('2024-06-04 18:40:54.546585088')],
  [Timestamp('2024-06-04 18:44:45.635715840'),
   Timestamp('2024-06-04 18:44:47.934376960')],
  [Timestamp('2024-06-04 19:07:00.671313920'),
   Timestamp('2024-06-04 19:07:02.590368')],
  [Timestamp('2024-06-04 19:17:46.235107072'),
   Timestamp('2024-06-04 19:17:48.553856768')],
  [Timestamp('2024-06-05 11:59:43.706454016'),
   Timestamp('2024-06-05 11:59:45.705260032')]],
 [[Timestamp('2024-06-04 18:34:58.318092800'),
   Timestamp('2024-06-04 18:35:00.416941056')],
  [Timestamp('2024-06-04 18:38:16.373419008'),
   Timestamp('2024-06-04 18:38:19.391865856')]],
 [[Timestamp('2024-06-04 18:32:56.440315136'),
   T

In [155]:
len(extracted_eye_data[0][0]['Timestamp'])

78

In [130]:
motion_data['Timestamp'][9719]

Timestamp('2024-06-04 18:34:53.860209920')

In [ ]:
# 指定された範囲のデータをフィルタリング
circle_start_time = '2024-06-04 16:00:31'
circle_end_time = '2024-06-04 16:00:34'
cross_start_time = '2024-06-04 16:10:55'
cross_end_time = '2024-06-04 16:10:58'
tri_start_time = '2024-06-04 16:31:37'
tri_end_time = '2024-06-04 16:31:40'
check_start_time = '2024-06-04 16:46:43'
check_end_time = '2024-06-04 16:46:46'
circle = motion_data[(motion_data['Timestamp'] >= circle_start_time) & (motion_data['Timestamp'] <= circle_end_time)].reset_index(drop=True)
cross = motion_data[(motion_data['Timestamp'] >= cross_start_time) & (motion_data['Timestamp'] <= cross_end_time)].reset_index(drop=True)
tri = motion_data[(motion_data['Timestamp'] >= tri_start_time) & (motion_data['Timestamp'] <= tri_end_time)].reset_index(drop=True)
check = motion_data[(motion_data['Timestamp'] >= check_start_time) & (motion_data['Timestamp'] <= check_end_time)].reset_index(drop=True)

In [ ]:
Th = 3.5

In [ ]:
segx = mr.spring_ogawa(motion_data['AccelerationX'], circle['AccelerationX'], Th)
segy = mr.spring_ogawa(motion_data['AccelerationY'], circle['AccelerationY'], Th)
segz = mr.spring_ogawa(motion_data['AccelerationZ'], circle['AccelerationZ'], Th)

In [ ]:
overlap_ranges = mr.combine_and_find_overlapping_segments(segx, segy, segz)

In [ ]:
overlap_ranges = mr.combine_and_find_all_overlapping_segments(segx, segy, segz)

In [ ]:
circle35 = overlap_ranges

In [ ]:
motion_data['Timestamp'][9760]

In [ ]:
cross3 = overlap_ranges

In [ ]:
tri35[36]

In [ ]:
check35

In [ ]:
motion_data['Timestamp'][66190]

In [ ]:
gesture_eye_data = []

In [ ]:
results = []
for start, end in circle35:
    a = [motion_data['Timestamp'][start], motion_data['Timestamp'][end]]
    results.append(a)

In [ ]:
# 経過時間を計算して格納するリスト
elapsed_times = []

# 各区間の経過時間を計算
for start_time, end_time in results:
    # タイムスタンプをdatetimeオブジェクトに変換
    start_dt = start_time
    end_dt = end_time

    # 経過時間を計算
    elapsed_time = (end_dt - start_dt).total_seconds()

    # 経過時間をリストに追加
    elapsed_times.append(elapsed_time)

# 経過時間のリストを表示
print("Elapsed times (seconds):", elapsed_times)

In [ ]:
gesture_eye_data = []

In [ ]:
for i in range(len(results)):
  gesture_eye_data.append(eye_data[(eye_data['Timestamp'] >= (results[i][0] - pd.Timedelta(seconds=5))) & (eye_data['Timestamp'] <= (results[i][1]) + pd.Timedelta(seconds=5))])

In [ ]:
len(gesture_eye_data)

In [ ]:
plt.figure(figsize=(72, 6))
plt.plot(gesture_eye_data[36]['Timestamp'], gesture_eye_data[36]['Gaze point X'].diff(), label='Gaze point X diff')  # 時系列データのプロット
plt.plot(gesture_eye_data[36]['Timestamp'], gesture_eye_data[36]['Gaze point Y'].diff(), label='Gaze point Y diff')  # 時系列データのプロット
# plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter right'].diff(), label='Pupil diameter right diff')  # 時系列データのプロット
# plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter left'].diff(), label='Pupil diameter left diff')  # 時系列データのプロット
plt.grid(True)  # グリッド表示
plt.legend()  # 凡例表示
plt.xticks(rotation=45)  # x軸のラベルを45度回転
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%M:%S:%f'))  # Timestampフォーマット設定
# ハイライト
plt.axvspan(results[36][0], results[36][1], color='yellow', alpha=0.3)
plt.show()

In [ ]:
for i in range(len(gesture_eye_data)):
    plt.figure(figsize=(72, 6))
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point X'].diff(), label='Gaze point X diff')  # 時系列データのプロット
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point Y'].diff(), label='Gaze point Y diff')  # 時系列データのプロット
    # plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter right'].diff(), label='Pupil diameter right diff')  # 時系列データのプロット
    # plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter left'].diff(), label='Pupil diameter left diff')  # 時系列データのプロット
    plt.grid(True)  # グリッド表示
    plt.legend()  # 凡例表示
    plt.xticks(rotation=45)  # x軸のラベルを45度回転
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%M:%S:%f'))  # Timestampフォーマット設定
    # ハイライト
    plt.axvspan(results[i][0], results[i][1], color='yellow', alpha=0.3)
    plt.show()

In [ ]:
def calculate_variance(data, start_time, end_time):
    # 指定された期間内のデータをフィルタリング
    filtered_data = data[(data['Timestamp'] >= start_time) & (data['Timestamp'] <= end_time)]
    # 差分の分散を計算
    gaze_x_var = filtered_data['Gaze point X'].diff().var()
    gaze_y_var = filtered_data['Gaze point Y'].diff().var()
    return gaze_x_var, gaze_y_var


In [ ]:
# 分散を格納する配列
variances_x = []
variances_y = []
# グラフのプロットと分散の計算
for i in range(len(gesture_eye_data)):
    plt.figure(figsize=(72, 6))
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point X'].diff(), label='Gaze point X diff')
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point Y'].diff(), label='Gaze point Y diff')
    plt.grid(True)
    plt.legend()
    plt.xticks(rotation=45)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%M:%S:%f'))

    # ハイライト区間の開始と終了時刻
    start_time = results[i][0]
    end_time = results[i][1]

    # ハイライト
    plt.axvspan(start_time, end_time, color='yellow', alpha=0.3)

    # 分散の計算
    gaze_x_var, gaze_y_var = calculate_variance(gesture_eye_data[i], start_time, end_time)

    # 分散を配列に格納
    variances_x.append(gaze_x_var)
    variances_y.append(gaze_y_var)

    # 分散の表示
    plt.title(f'Gaze point X variance: {gaze_x_var:.5f}, Gaze point Y variance: {gaze_y_var:.5f}')

    plt.show()

# 分散の配列を表示
print("Gaze point X variances:", variances_x)
print("Gaze point Y variances:", variances_y)

In [ ]:
np.sqrt(variances_x)

In [ ]:
np.sqrt(variances_y)

In [ ]:
for i in range(len(gesture_eye_data)):
    plt.figure(figsize=(72, 6))
    # plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point X'].diff(), label='Gaze point X diff')  # 時系列データのプロット
    # plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point Y'].diff(), label='Gaze point Y diff')  # 時系列データのプロット
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter right'].diff(), label='Pupil diameter right diff')  # 時系列データのプロット
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter left'].diff(), label='Pupil diameter left diff')  # 時系列データのプロット
    plt.grid(True)  # グリッド表示
    plt.legend()  # 凡例表示
    plt.xticks(rotation=45)  # x軸のラベルを45度回転
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%M:%S:%f'))  # Timestampフォーマット設定
    # ハイライト
    plt.axvspan(results[i][0], results[i][1], color='yellow', alpha=0.3)
    plt.show()